In [ ]:
from fenics import *
import numpy as np
from numpy import save
from numpy.random import random
import matplotlib.pyplot as plt
from mshr import *
import os
set_log_level(30)

In [ ]:
class PredatorPrey(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
    def F(self, b, x): assemble(self.L, tensor=b)
    def J(self, A, x): assemble(self.a, tensor=A)

In [ ]:
"""
Define random initial condition
"""
class IC(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 1.0*random() +0.25
        values[1] = 1.0*random() +0.25
    def value_shape(self): return (2,)

In [ ]:
def fenics_field_to_numpy_array(fenics_field, coordinates_size, step, field_name): 
    field_array = np.empty((0, coordinates_size), int)
    lista = []
    for val_x in range(0,coordinates_size):
        for val_y in range(0,coordinates_size):
            valor=fenics_field(val_x,val_y)
            lista.append(valor)

            if len(lista) == coordinates_size:
                field_array = np.append(field_array, np.array([lista]), axis=0)
                if val_x < coordinates_size:
                    lista = []
    N_field = "matrix_"  + field_name +"_"+ str(step) + ".txt"
    np.savetxt(N_field, field_array, delimiter="\t")
    return None

In [ ]:
def grafica(P, N, t):
    plt.figure(figsize=(11, 4))
    
    plt.subplot(1, 2, 1)
    p1 = plot(P)
    p1.set_cmap("seismic")
    plt.title("$P(t={})$".format(t))
    plt.colorbar(p1) 
    
    plt.subplot(1, 2, 2)
    p1 = plot(N)
    p1.set_cmap("gray")
    plt.title("$N(t={})$".format(t))
    plt.colorbar(p1) 
    
    plt.tight_layout(pad=-1.0)
    plt.show()

In [ ]:
def mesh():
    # Define finite diference mesh
    p0 = Point(0.0, 0.0)
    p1 = Point(70,70)
    square = Rectangle(p0, p1)  
    mesh = RectangleMesh(p0, p1, 100,100,"right/left")
    return mesh

In [ ]:
def function_space(mesh):
    # Work in function space, test and trial functions
    U = FiniteElement("CG", mesh.ufl_cell(), 2)
    W = FunctionSpace(mesh, U * U)
    du   = TrialFunction(W)
    q, p = TestFunctions(W)
    w = Function(W)
    w0 =  Function(W)
    return W,du,q,p,w,w0

In [ ]:
W,du,q,p,w,w0 = function_space(mesh())
# Split mixed functions
dN, dP = split(du)
N, P = split(w)
N0, P0 = split(w0)

In [ ]:
# Apply random inital condition
w_init = IC(element=W.ufl_element(), degree=2)
w.interpolate(w_init)
w0.interpolate(w_init)

In [ ]:
# Time parameters
dt = 5 # step
T = 1000 # end step
t = 0  # zero time

# Set parameters
D_N = 0.015
D_P = 1.0
alpha = 0.7
beta = 0.25
gamma = 0.5

In [ ]:

# writing PDE system in weak form
L0 = N*q -N0*q +D_N*inner(grad(N), grad(q))*dt - alpha*N*(1-N)*(N+P)*q*dt + N*P*q*dt
L1 = P*p -P0*p +D_P*inner(grad(P), grad(p))*dt + beta*P*(N+P)*p*dt - gamma*N*P*p*dt
L  = (L0 + L1) *dx

# Compute directional derivative about u in the direction of du
a = derivative(L, w, du)
problem = PredatorPrey(a, L)
solver = NewtonSolver()
solver.parameters["linear_solver"] = "lu"
solver.parameters["convergence_criterion"] = "incremental"
solver.parameters["relative_tolerance"] = 1e-1

In [ ]:
grafica(P0, N0, t)

In [ ]:
os.chdir('/home/erick/Desktop/FEniCS/FEniCS-projects/diffution systems/predator-prey/fields')
while t < T:
    t += dt
    w0.vector()[:] = w.vector()
    solver.solve(problem, w.vector())
    N,P = w.split()

    dofmap = W.dofmap()
    dof_x = W.tabulate_dof_coordinates().reshape(W.dim(),mesh().geometry().dim())
    w_vector = w.vector().get_local()
    fenics_field_to_numpy_array(P, 70, t, "P" )
    fenics_field_to_numpy_array(N, 70, t, "N" )
    
    print("t=", t)
    grafica(P, N, t)